In [4]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [5]:
state_to_stfips_path ="/project/biocomplexity/COVID-19_commons/2019-nCoV_stat-analyses/misc_data/stfips_to_state.csv"
state_to_stfips_df = pd.read_csv(state_to_stfips_path, header=None, dtype={"2":str})
state_to_stfips_df[2] = state_to_stfips_df[2].astype(str).str.zfill(2)
state_to_stfips = state_to_stfips_df.set_index(0)[2].to_dict()
state_to_stfips = state_to_stfips_df.set_index(0)[2].to_dict()
state_to_state_abbr = state_to_stfips_df.set_index(0)[1].to_dict()
stfips_to_state_abbr = state_to_stfips_df.set_index(2)[1].to_dict()
abbr_to_stfips = state_to_stfips_df.set_index(1)[2].to_dict()

In [6]:
def get_state_time_series_outbreakinfo(state, strain="B.1.617.2"):
#     print(state, strain)
    time.sleep(0.5)
    base_url = "https://api.outbreak.info/genomics/prevalence-by-location"

    headers = {"Accept": "application/json, text/plain, */*", 
           "Accept-Encoding": "gzip, deflate, br"}
    
    if state in ["United States", "US", "USA", "ALL"]:
        form_data = {"pangolin_lineage": strain,
         "location_id": "USA",
         "cumulative": "false",
         "timestamp": 448911}
    else:
        if len(state) > 2:
            state = state_to_state_abbr[state]
        else:
            state = stfips_to_state_abbr[state]
        form_data = {"pangolin_lineage": strain,
#              "division": state,
             "location_id": f"USA_US-{state}",
             "cumulative": "false", 
             "timestamp": 448911}
    response = json.loads(requests.get(base_url, headers = headers, data=form_data).text)["results"]
    try:
        x = pd.DataFrame(response)
        x.date = pd.to_datetime(x.date)
        x.date = x.date.apply(lambda row: pd.to_datetime(row, format="%Y-%dm-%d").date())
    except:
        columns = ['date', 'total_count', 'lineage_count', 'total_count_rolling',
       'lineage_count_rolling', 'proportion', 'proportion_ci_lower',
       'proportion_ci_upper']
        null_df = {col:[] for col in columns}
        x = pd.DataFrame(null_df)
    return x

In [7]:
vocs = ['B.1.1.7',
        'B.1.351',
        'P.1',
        'B.1.617.1',
        'B.1.617.2',
        'B.1.617.3',
        'AY.3',
        'B.1.427',
        'B.1.429',
        'B.1.526',
        'B.1.526.1',
        'B.1.526.2']

states = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
       '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53',
       '54', '55', '56', '66', '69', '72', '78']

In [ ]:
dfs = []

for state in states:
    for variant in vocs:
        data = get_state_time_series_outbreakinfo(state)
        print(data.date.max())
        data["fips"] = state
        data["lineage"] = variant
        dfs.append(data)
#     break

len(dfs)

In [11]:
pd.concat(dfs).to_csv("/project/biocomplexity/COVID-19_commons/data/variants/outbreak_info_variants_states_long.csv", index=False)

In [12]:
pd.read_csv("/project/biocomplexity/COVID-19_commons/data/variants/outbreak_info_variants_states_long.csv")

,date,total_count,lineage_count,total_count_rolling,lineage_count_rolling,proportion,proportion_ci_lower,proportion_ci_upper,fips,lineage
0,2021-04-16,22.0,1.0,22.000000,1.000000,0.045455,0.004949,0.193437,1,B.1.1.7
1,2021-04-17,3.0,0.0,12.500000,0.500000,0.040000,0.000040,0.185306,1,B.1.1.7
2,2021-04-18,6.0,0.0,10.333333,0.333333,0.032258,0.000048,0.217196,1,B.1.1.7
3,2021-04-19,10.0,1.0,10.250000,0.500000,0.048780,0.000048,0.217196,1,B.1.1.7
4,2021-04-20,13.0,0.0,10.800000,0.400000,0.037037,0.000044,0.199995,1,B.1.1.7
...,...,...,...,...,...,...,...,...,...,...
68587,2021-07-23,12.0,12.0,11.000000,10.428571,0.948052,0.647024,0.990009,56,B.1.526.2
68588,2021-07-24,9.0,9.0,11.571429,11.285714,0.975309,0.671504,0.990856,56,B.1.526.2
68589,2021-07-25,1.0,1.0,10.857143,10.571429,0.973684,0.800005,0.999956,56,B.1.526.2
68590,2021-07-26,8.0,7.0,9.857143,9.428571,0.956522,0.618685,0.988988,56,B.1.526.2
